In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os

os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search,
    obtain_SWNUNetwork_input,
)

Rumours

In [4]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0


In [6]:
sbert_embeddings.shape

torch.Size([5568, 384])

In [69]:
shift = 3
window_size = 5
n = 6

history_length = shift * n + (window_size - shift)
print(history_length)

20


In [10]:
x_data = obtain_SWNUNetwork_input(
    method="umap",
    dimension=30,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    k=history_length,
    time_feature="time_encoding",
    standardise_method=None,
    add_time_in_path=False,
)

x_data[0].shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


torch.Size([5568, 20, 415])

Below is what we do in `.get_torch_path_for_SeqSigNet()` (a straight copy and paste and passing in `x_data[0]` as `swnu_path`)

In [70]:
test = torch.stack(
    [x_data[0][:, (i * shift) : (i * shift + window_size), :] for i in range(n)],
    dim=3,
)

`test` has dimensions `[batch, window_size, channels, units]`

In [71]:
test.shape

torch.Size([5568, 5, 415, 6])

In [59]:
x_data[0][1]

tensor([[ 9.9126, 10.1758,  2.8591,  ..., -0.1027, -0.0346,  0.0936],
        [10.2004,  9.8124,  3.0175,  ..., -0.1027, -0.0346,  0.0936],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1027, -0.0346,  0.0936],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1027, -0.0346,  0.0936],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1027, -0.0346,  0.0936],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1027, -0.0346,  0.0936]])

In [63]:
test[1, :, :, 3]

tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.1027, -0.0346,  0.0936],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1027, -0.0346,  0.0936],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1027, -0.0346,  0.0936],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1027, -0.0346,  0.0936],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.1027, -0.0346,  0.0936]])

Below is how we obtain the embeddings in SeqSigNet (remember that `x_data[1]` gives the `input_channels`)

In [79]:
test[:, 0, (x_data[1] + 1) :, 0]

tensor([[ 0.0648,  0.0094, -0.0334,  ..., -0.0119,  0.0184,  0.0019],
        [-0.0293, -0.0539, -0.0399,  ..., -0.1027, -0.0346,  0.0936],
        [-0.0319,  0.0551, -0.0240,  ...,  0.1048,  0.0090,  0.0933],
        ...,
        [-0.0022,  0.0809, -0.0992,  ..., -0.0370,  0.0329, -0.0239],
        [-0.0313,  0.0355,  0.0073,  ..., -0.0722,  0.0361, -0.0135],
        [ 0.0112,  0.0805, -0.0411,  ..., -0.0098, -0.0139,  0.0893]])

Below are the embeddings - we can see they're the same

In [68]:
sbert_embeddings

tensor([[ 0.0648,  0.0094, -0.0334,  ..., -0.0119,  0.0184,  0.0019],
        [-0.0293, -0.0539, -0.0399,  ..., -0.1027, -0.0346,  0.0936],
        [-0.0319,  0.0551, -0.0240,  ...,  0.1048,  0.0090,  0.0933],
        ...,
        [-0.0022,  0.0809, -0.0992,  ..., -0.0370,  0.0329, -0.0239],
        [-0.0313,  0.0355,  0.0073,  ..., -0.0722,  0.0361, -0.0135],
        [ 0.0112,  0.0805, -0.0411,  ..., -0.0098, -0.0139,  0.0893]],
       dtype=torch.float64)

In [78]:
torch.equal(test[:, 0, (x_data[1] + 1) :, 0].float(), sbert_embeddings.float())

True